# Stack and Queue Interview Workbook


Stacks, queues, and deques sit at the heart of many interview classics.
This notebook curates every stack and queue exercise in the repository so you can revise the prompt,
walk through a representative example, review the guiding idea, and study the C++ implementation in one place.


## Stack Fundamentals and Simulation


Build intuition for stack-based validation and state simulation problems before moving on to advanced patterns.


### Validate Balanced Parentheses

**Problem statement:** Given a string containing the characters '(', ')', '{', '}', '[' and ']', determine if the input string is valid. A string is valid if open brackets are closed by the same type of bracket and in the correct order.

**Example:**
```
s = "{[()]}"
Output: true
```
**Explanation:** The algorithm pushes opening brackets onto a stack and matches them against closing brackets using a lookup map. Any mismatch or premature closing bracket invalidates the string, and the stack must be empty when processing ends.

**Approach highlights:**
* Create a mapping from each closing bracket to the expected opening bracket.
* Iterate through the string, pushing every opening bracket onto the stack as it appears.
* For a closing bracket, ensure the stack is non-empty and the top element matches the required opening bracket; otherwise return false immediately.
* After processing all characters, confirm the stack is empty to verify that every opening bracket was closed.

**Complexity:** Time O(n), Space O(n).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution {
public:
    bool isValid(string s)
    {
        map<char,char> mapping;
        mapping[')'] = '(';
        mapping['}'] = '{';
        mapping[']'] = '[';
        stack<char> dummy;
        for(int i = 0;i < s.length();i++)
        {
            if(s[i] == '(' || s[i] == '{' || s[i] == '[')
            {
                dummy.push(s[i]);
            }
            else
            {
                if(dummy.size() == 0)
                {
                    return false;
                }
                if(dummy.top() == mapping[s[i]])
                {
                    dummy.pop();
                }
                else
                {
                    return false;
                }
            }
        }
        if(dummy.size() == 0)
        {
            return true;
        }
        else
        {
            return false;
        }
    }
};


### Design a Minimum-Tracking Stack

**Problem statement:** Implement a stack that supports push, pop, top, and retrieving the minimum element in constant time.

**Example:**
```
Operations:
push(2)
push(0)
push(3)
push(0)
getMin() -> 0
pop()
getMin() -> 0
pop()
getMin() -> 0
pop()
getMin() -> 2
```
**Explanation:** Each stack node records both its value and the minimum seen up to that point. The current minimum is always stored alongside the top node, allowing each operation to run in constant time.

**Approach highlights:**
* Store pairs of (value, current minimum) inside the stack structure.
* When pushing, compute the new minimum as the smaller of the incoming value and the previous minimum, then push the pair.
* Pop operations simply discard the top pair, automatically exposing the previous minimum.
* top() returns the latest value, and getMin() returns the minimum stored with that node.

**Complexity:** Time O(1) per operation, Space O(n).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class MinStack
{
private:
    stack<pair<int, int>> st;

public:
    MinStack() = default;

    void push(int val)
    {
        int currentMin = st.empty() ? val : min(val, st.top().second);
        st.emplace(val, currentMin);
    }

    void pop()
    {
        if (!st.empty())
        {
            st.pop();
        }
    }

    int top()
    {
        return st.top().first;
    }

    int getMin()
    {
        return st.top().second;
    }
};


### Simulate Asteroid Collisions

**Problem statement:** Given an array of integers representing asteroids moving along a line, where the absolute value is the size and the sign is the direction (positive moves right, negative moves left), return the state after all collisions. Smaller asteroids explode; equal-sized asteroids destroy each other.

**Example:**
```
asteroids = [5, 10, -5]
Output: [5, 10]
```
**Explanation:** The implementation tracks indices of right-moving asteroids on a stack. When a left-moving asteroid arrives, it repeatedly collides with right-moving asteroids until it either explodes, destroys an equal-sized asteroid, or survives because no right-moving rocks remain in front of it.

**Approach highlights:**
* Traverse the list and push indices of right-moving asteroids onto a stack.
* When a left-moving asteroid is encountered, pop any smaller right-moving asteroids from the stack and mark them as destroyed.
* If an asteroid of equal magnitude is found, mark both as destroyed and remove the right-moving asteroid from the stack.
* Collect every asteroid still marked as alive into the output list at the end of the scan.

**Complexity:** Time O(n), Space O(n).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:
    vector<int> asteroidCollision(vector<int>& asteroids)
    {
        int n = asteroids.size();
        vector<int> ans;
        stack<int> s;
        vector<int> possible(n,1);
        for(int i = 0;i < n;i++)
        {
            if(asteroids[i] < 0)
            {
                while(!s.empty() && asteroids[s.top()] < abs(asteroids[i]))
                {
                    possible[s.top()] = 0;
                    s.pop();
                }
                if(!s.empty())
                {
                    possible[i] = 0;
                    if(asteroids[s.top()] == abs(asteroids[i]))
                    {
                        possible[s.top()] = 0;
                        s.pop();
                    }
                }
            }
            else
            {
                s.push(i);
            }
        }
        for(int i = 0;i < n;i++)
        {
            if(possible[i])
            {
                ans.push_back(asteroids[i]);
            }
        }
        return ans;
    }
};


## Monotonic Stack for Next/Previous Comparisons


Monotonic stacks efficiently answer next and previous greater-or-smaller queries and power several greedy optimisations.


### Next Greater Element I

**Problem statement:** For each element in nums1, find the next greater element appearing to its right in nums2. If it does not exist, return -1.

**Example:**
```
nums1 = [4, 1, 2]
nums2 = [1, 3, 4, 2]
Output: [-1, 3, -1]
```
**Explanation:** Scanning nums2 from right to left maintains a decreasing stack of candidates. A map records each value's next greater element, and a final pass over nums1 replaces each query with the mapped answer.

**Approach highlights:**
* Initialise a stack with a sentinel -1 to represent missing next greater elements.
* Iterate through nums2 from right to left, popping any stack values less than the current element.
* After the pops, the stack top holds the next greater value (or -1); record it in a map and push the current element.
* Build the answer for nums1 by looking up each value's mapped next greater element.

**Complexity:** Time O(n + m), Space O(m).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:
    vector<int> nextGreaterElement(vector<int>& nums1, vector<int>& nums2)
    {
        int n = nums1.size();
        int m = nums2.size();
        vector<int> ans(n);
        map<int,int> mapping;
        stack<int> s;
        s.push(-1);
        for(int i = m - 1;i >= 0;i--)
        {
            while(s.top() != -1 && s.top() < nums2[i])
            {
                s.pop();
            }
            mapping[nums2[i]] = s.top();
            s.push(nums2[i]);
        }
        for(int i = 0;i < n;i++)
        {
            ans[i] = mapping[nums1[i]];
        }
        return ans;
    }
};


### Next Greater Element II (Circular Array)

**Problem statement:** Given a circular array nums, compute the next greater element for every index, wrapping around to the beginning of the array when necessary.

**Example:**
```
nums = [1, 2, 1]
Output: [2, -1, 2]
```
**Explanation:** The algorithm preloads a decreasing stack with a single backward pass, then performs a second pass to assign answers, continually discarding values that are not greater than the current element.

**Approach highlights:**
* Traverse the array from right to left once to seed the stack with elements in decreasing order.
* Perform a second backward traversal, popping stack elements that are less than or equal to the current value.
* If the stack is non-empty, record its top as the next greater element for the current index; otherwise record -1.
* Push the current value so earlier elements can observe it when their turn arrives.

**Complexity:** Time O(n), Space O(n).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:
    vector<int> nextGreaterElements(vector<int>& nums)
    {
        int n = nums.size();
        vector<int> ans(n,-1);
        stack<int> s;
        for(int i = n - 1;i >= 0;i--)
        {
            while(!s.empty() && s.top() <= nums[i])
            {
                s.pop();
            }
            s.push(nums[i]);
        }
        for(int i = n - 1;i >= 0;i--)
        {
            while(!s.empty() && s.top() <= nums[i])
            {
                s.pop();
            }
            if(!s.empty())
            {
                ans[i] = s.top();
            }
            s.push(nums[i]);
        }
        return ans;
    }
};


### Previous Smaller Element

**Problem statement:** For each position in an array, find the value of the nearest smaller element to its left. If no such element exists, return -1 for that index.

**Example:**
```
nums = [4, 5, 2, 10, 8]
Output: [-1, 4, -1, 2, 2]
```
**Explanation:** A monotonic increasing stack keeps track of potential previous candidates. Whenever a new number arrives, larger or equal values are popped until the stack exposes the nearest smaller value.

**Approach highlights:**
* Maintain a stack that stores values in strictly increasing order from bottom to top.
* For each number, pop stack entries that are greater than or equal to the current value.
* After the pops, if the stack is non-empty, its top is the nearest smaller element; otherwise record -1.
* Push the current value for future indices to reference.

**Complexity:** Time O(n), Space O(n).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:
    vector<int> previousSmallerElements(vector<int>& nums)
    {
        int n = nums.size();
        vector<int> ans(n,-1);
        stack<int> s;
        for(int i = 0;i < n;i++)
        {
            while(!s.empty() && s.top() >= nums[i])
            {
                s.pop();
            }
            if(!s.empty())
            {
                ans[i] = s.top();
            }
            s.push(nums[i]);
        }
        return ans;
    }
};


### Online Stock Span

**Problem statement:** Design a data structure that, for each incoming stock price, returns the number of consecutive days (including today) the price has been less than or equal to today's price.

**Example:**
```
prices = [100, 80, 60, 70, 60, 75, 85]
Output: [1, 1, 1, 2, 1, 4, 6]
```
**Explanation:** The stack stores pairs of (price, span). When a new price arrives, the structure collapses any weaker prices beneath it, accumulating their spans so the final answer can be returned in constant time.

**Approach highlights:**
* For each query price, start its span at one (covering the current day).
* While the stack top has a price less than or equal to the new price, pop it and add its span to the running total.
* Push the current price along with the aggregated span onto the stack.
* Return the computed span as the answer for the query.

**Complexity:** Time O(n), Space O(n).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class StockSpanner
{
private:
    stack<pair<int, int>> st;

public:
    StockSpanner() = default;

    int next(int price)
    {
        int span = 1;
        while (!st.empty() && st.top().first <= price)
        {
            span += st.top().second;
            st.pop();
        }
        st.emplace(price, span);
        return span;
    }
};


### Remove K Digits for the Smallest Number

**Problem statement:** Given a non-negative integer num represented as a string and an integer k, remove k digits from the number so that the new number is the smallest possible.

**Example:**
```
num = "1432219", k = 3
Output: 1219
```
**Explanation:** A monotonic increasing stack of digits guarantees that any higher digit preceding a smaller digit will be removed first. Leading zeros are skipped, and leftover removals trim digits from the end.

**Approach highlights:**
* Iterate through each digit, skipping leading zeros when the stack is empty.
* While k is positive and the stack top digit is greater than the current digit, pop the stack and decrement k.
* Push the current digit, ensuring the structure stays non-decreasing.
* If removals remain after scanning, pop from the stack until k reaches zero, then build the result in reverse order.

**Complexity:** Time O(n), Space O(n).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:
    string removeKdigits(string num, int k)
    {
        stack<char> s;
        for(char c : num)
        {
            if(c == '0' && s.empty())
            {
                continue; // Skip leading zeros
            }
            while(k > 0 && !s.empty() && s.top() > c)
            {
                s.pop();
                k--;
            }
            s.push(c);
        }
        while(k > 0 && !s.empty())
        {
            s.pop();
            k--;
        }
        string ans = "";
        while(!s.empty())
        {
            ans += s.top();
            s.pop();
        }
        reverse(ans.begin(),ans.end());
        return ans.empty() ? "0" : ans;
    }
};


## Monotonic Stack Range Aggregations


These problems compute contributions from every index by measuring how far it can extend before a smaller or greater value blocks it.


### Largest Rectangle in a Histogram

**Problem statement:** Given an array of bar heights representing a histogram, return the area of the largest rectangle that can be formed within the histogram.

**Example:**
```
heights = [2, 1, 5, 6, 2, 3]
Output: 10
```
**Explanation:** The solution records each bar's nearest smaller element on both sides. The width of the rectangle contributed by each bar is determined by these boundaries, allowing the algorithm to maximise area efficiently.

**Approach highlights:**
* Sweep from left to right with a stack to compute the index of the next smaller element for every bar.
* During the same pass, capture the previous smaller index for each bar once the stack is non-empty.
* For each bar, calculate the rectangle width using the two boundaries and update the maximum area.

**Complexity:** Time O(n), Space O(n).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:
    int largestRectangleArea(vector<int>& heights)
    {
        int n = heights.size();
        vector<int> nse(n,n);
        vector<int> psee(n,-1);
        stack<int> s;
        for(int i = 0;i < n;i++)
        {
            while(!s.empty() && heights[s.top()] > heights[i])
            {
                nse[s.top()] = i;
                s.pop();
            }
            if(!s.empty())
            {
                psee[i] = s.top();
            }
            s.push(i);
        }
        int maxArea = -1;
        for(int i = 0;i < n;i++)
        {
            int width = nse[i] - psee[i] - 1;
            maxArea = max(maxArea,heights[i]*width);
        }
        return maxArea;
    }
};


### Maximal Rectangle in a Binary Matrix

**Problem statement:** Given a binary matrix filled with 0s and 1s, find the largest rectangular area containing only 1s.

**Example:**
```
matrix =
[
  ["1","0","1","0","0"],
  ["1","0","1","1","1"],
  ["1","1","1","1","1"],
  ["1","0","0","1","0"]
]
Output: 6
```
**Explanation:** Each row is treated as the base of a histogram whose heights count consecutive 1s above it. The histogram solver from the previous problem is then applied to every row to keep track of the best rectangle.

**Approach highlights:**
* Precompute column-wise heights representing consecutive ones ending at each row.
* For each row, feed the height array into the histogram routine to compute the largest rectangle that uses that row as its base.
* Track the overall maximum area across all rows and return it at the end.

**Complexity:** Time O(rows * cols), Space O(rows * cols).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
private:
    int largestRectangleArea(vector<int>& heights)
    {
        int n = heights.size();
        vector<int> nse(n,n);
        vector<int> psee(n,-1);
        stack<int> s;
        for(int i = 0;i < n;i++)
        {
            while(!s.empty() && heights[s.top()] > heights[i])
            {
                nse[s.top()] = i;
                s.pop();
            }
            if(!s.empty())
            {
                psee[i] = s.top();
            }
            s.push(i);
        }
        int maxArea = -1;
        for(int i = 0;i < n;i++)
        {
            int width = nse[i] - psee[i] - 1;
            maxArea = max(maxArea,heights[i]*width);
        }
        return maxArea;
    }
public:
    int maximalRectangle(vector<vector<char>>& matrix)
    {
        int row = matrix.size();
        int col = matrix[0].size();
        vector<vector<int>> prefixHeight(row,vector<int>(col,0));
        for(int i = 0;i < col;i++)
        {
            prefixHeight[0][i] = matrix[0][i] - '0';
        }
        for(int j = 0;j < col;j++)
        {
            for(int i = 1;i < row;i++)
            {
                if(matrix[i][j] == '0')
                {
                    prefixHeight[i][j] = 0;
                }
                else
                {
                    prefixHeight[i][j] = prefixHeight[i - 1][j] + matrix[i][j] - '0';
                }
            }
        }
        int maxArea = -1;
        for(int i = 0;i < row;i++)
        {
            maxArea = max(maxArea,largestRectangleArea(prefixHeight[i]));
        }
        return maxArea;
    }
};


### Sum of Subarray Minimums

**Problem statement:** For an integer array, compute the sum of the minimum element of every contiguous subarray. Return the result modulo 1e9 + 7.

**Example:**
```
arr = [3, 1, 2, 4]
Output: 17
```
**Explanation:** Two monotonic stacks locate the next and previous smaller elements for each index. These spans count how many subarrays adopt the current value as their minimum contribution.

**Approach highlights:**
* Find the next strictly smaller element index for every position using a monotonic stack.
* Find the previous smaller-or-equal index with another stack to avoid double counting.
* Multiply the distances to the boundaries to determine the number of subarrays where the element is the minimum.
* Accumulate the contributions under modulo arithmetic to build the final answer.

**Complexity:** Time O(n), Space O(n).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

int mod = 1e9 + 7;

class Solution
{
private:
    vector<int> findNSE(vector<int>& arr)
    {
        int n = arr.size();
        stack<int> s;
        vector<int> ans(n,n);
        for(int i = n - 1;i >= 0;i--)
        {
            while(!s.empty() && arr[s.top()] >= arr[i])
            {
                s.pop();
            }
            if(!s.empty())
            {
                ans[i] = s.top();
            }
            s.push(i);
        }
        return ans;
    }
    vector<int> findPSEE(vector<int>& arr)
    {
        int n = arr.size();
        stack<int> s;
        vector<int> ans(n,-1);
        for(int i = 0;i < n;i++)
        {
            while(!s.empty() && arr[s.top()] > arr[i])
            {
                s.pop();
            }
            if(!s.empty())
            {
                ans[i] = s.top();
            }
            s.push(i);
        }
        return ans;
    }
public:
    int sumSubarrayMins(vector<int>& arr)
    {
        int n = arr.size();
        vector<int> nse = findNSE(arr);
        vector<int> psee = findPSEE(arr);
        int ans = 0;
        for(int i = 0;i < n;i++)
        {
            int left = i - psee[i];
            int right = nse[i] - i;
            ans = (ans + (left*right*1LL*arr[i])%mod)%mod;
        }
        return ans;
    }
};


### Sum of Subarray Maximums

**Problem statement:** Given an integer array, compute the sum of the maximum element of every contiguous subarray. Return the result modulo 1e9 + 7.

**Example:**
```
arr = [1, 3, 2]
Output: 15
```
**Explanation:** The implementation mirrors the minimum-sum technique but inverts the comparisons to locate larger blocking elements, counting how many subarrays treat the current value as their maximum.

**Approach highlights:**
* Use a decreasing stack to record each index's next greater element (strictly greater).
* Use another stack to record the previous greater-or-equal index, preventing over-counting identical values.
* Combine the left and right spans to count the subarrays where the element is the maximum.
* Add each weighted contribution under the required modulo to obtain the final total.

**Complexity:** Time O(n), Space O(n).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

int mod = 1e9 + 7;

class Solution
{
private:
    vector<int> findNGE(vector<int>& arr)
    {
        int n = arr.size();
        stack<int> s;
        vector<int> ans(n,n);
        for(int i = n - 1;i >= 0;i--)
        {
            while(!s.empty() && arr[s.top()] <= arr[i])
            {
                s.pop();
            }
            if(!s.empty())
            {
                ans[i] = s.top();
            }
            s.push(i);
        }
        return ans;
    }
    vector<int> findPGEE(vector<int>& arr)
    {
        int n = arr.size();
        stack<int> s;
        vector<int> ans(n,-1);
        for(int i = 0;i < n;i++)
        {
            while(!s.empty() && arr[s.top()] < arr[i])
            {
                s.pop();
            }
            if(!s.empty())
            {
                ans[i] = s.top();
            }
            s.push(i);
        }
        return ans;
    }
public:
    int sumSubarrayMaxs(vector<int>& arr)
    {
        int n = arr.size();
        vector<int> nge = findNGE(arr);
        vector<int> pgee = findPGEE(arr);
        int ans = 0;
        for(int i = 0;i < n;i++)
        {
            int left = i - pgee[i];
            int right = nge[i] - i;
            ans = (ans + (left*right*1LL*arr[i])%mod)%mod;
        }
        return ans;
    }
};


### Sum of Subarray Ranges

**Problem statement:** For an integer array, return the sum of the ranges (maximum minus minimum) of every contiguous subarray. Use modulo arithmetic as defined in the implementation.

**Example:**
```
nums = [1, 2, 3]
Output: 4
```
**Explanation:** Range sums equal the total contribution of maxima minus the total contribution of minima. Four monotonic passes compute the necessary next and previous smaller/greater indices to evaluate both parts.

**Approach highlights:**
* Compute next and previous smaller indices to measure how often each element acts as a subarray minimum.
* Compute next and previous greater indices to measure how often each element acts as a subarray maximum.
* Accumulate the weighted minima and maxima separately using modulo arithmetic.
* Return the difference between the two totals to obtain the sum of ranges.

**Complexity:** Time O(n), Space O(n).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

const long long MOD = 1e18 + 7;

class Solution
{
private:
    vector<int> findNSE(vector<int>& arr)
    {
        int n = arr.size();
        stack<int> s;
        vector<int> ans(n,n);
        for(int i = n - 1;i >= 0;i--)
        {
            while(!s.empty() && arr[s.top()] > arr[i])
            {
                s.pop();
            }
            if(!s.empty())
            {
                ans[i] = s.top();
            }
            s.push(i);
        }
        return ans;
    }
    vector<int> findPSEE(vector<int>& arr)
    {
        int n = arr.size();
        stack<int> s;
        vector<int> ans(n,-1);
        for(int i = 0;i < n;i++)
        {
            while(!s.empty() && arr[s.top()] >= arr[i])
            {
                s.pop();
            }
            if(!s.empty())
            {
                ans[i] = s.top();
            }
            s.push(i);
        }
        return ans;
    }
    vector<int> findNGE(vector<int>& arr)
    {
        int n = arr.size();
        stack<int> s;
        vector<int> ans(n,n);
        for(int i = n - 1;i >= 0;i--)
        {
            while(!s.empty() && arr[s.top()] < arr[i])
            {
                s.pop();
            }
            if(!s.empty())
            {
                ans[i] = s.top();
            }
            s.push(i);
        }
        return ans;
    }
    vector<int> findPGEE(vector<int>& arr)
    {
        int n = arr.size();
        stack<int> s;
        vector<int> ans(n,-1);
        for(int i = 0;i < n;i++)
        {
            while(!s.empty() && arr[s.top()] <= arr[i])
            {
                s.pop();
            }
            if(!s.empty())
            {
                ans[i] = s.top();
            }
            s.push(i);
        }
        return ans;
    }

public:
    long long subArrayRanges(vector<int>& nums)
    {
        int n = nums.size();
        vector<int> nse_arr = findNSE(nums);
        vector<int> nge_arr = findNGE(nums);
        vector<int> psee_arr = findPSEE(nums);
        vector<int> pgee_arr = findPGEE(nums);
        long long sumMin = 0;
        long long sumMax = 0;
        for(int i = 0; i < n; i++)
        {
            long long left_count = i - psee_arr[i];
            long long right_count = nse_arr[i] - i;
            sumMin = (sumMin + (left_count*right_count*1LL*nums[i])%MOD)%MOD;
        }

        for(int i = 0; i < n; i++)
        {
            long long left_count = i - pgee_arr[i];
            long long right_count = nge_arr[i] - i;
            sumMax = (sumMax + (left_count*right_count*1LL*nums[i])%MOD)%MOD;
        }

        return (sumMax - sumMin + MOD) % MOD;
    }
};


## Queues, Deques, and Two-Pointer Optimisations


Windowing problems and elimination games often rely on specialised queue-like structures or bidirectional scans to stay linear.


### Sliding Window Maximum

**Problem statement:** Given an array nums and an integer k, return the maximum element in every contiguous subarray of size k.

**Example:**
```
nums = [1, 3, -1, -3, 5, 3, 6, 7], k = 3
Output: [3, 3, 5, 5, 6, 7]
```
**Explanation:** A deque of indices stores candidates in decreasing order. Out-of-window indices are removed from the front, and any smaller trailing indices are pruned so the deque front always holds the current maximum.

**Approach highlights:**
* Iterate through the array once, removing indices that fall out of the sliding window from the front of the deque.
* While the deque's back holds values less than or equal to the current element, pop them to maintain decreasing order.
* Push the current index, and once the first full window is formed, record the value at the deque front as the maximum.

**Complexity:** Time O(n), Space O(k).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:
    vector<int> maxSlidingWindow(vector<int>& nums,int k)
    {
        int n = nums.size();
        vector<int> ans;
        deque<int> d;
        for(int i = 0;i < n;i++)
        {
            if(!d.empty() && d.front() == (i - k))
            {
                d.pop_front();
            }
            while(!d.empty() && nums[d.back()] <= nums[i])
            {
                d.pop_back();
            }
            d.push_back(i);
            if(i >= (k - 1))
            {
                ans.push_back(nums[d.front()]);
            }
        }
        return ans;
    }
};


### Trapping Rain Water

**Problem statement:** Given non-negative integers representing an elevation map, compute how much water it can trap after raining.

**Example:**
```
height = [0, 1, 0, 2, 1, 0, 1, 3, 2, 1, 2, 1]
Output: 6
```
**Explanation:** The code precomputes suffix maxima and tracks a running prefix maximum while scanning from left to right. Each bar can trap water equal to the difference between the smaller of the two maxima and its own height, floored at zero.

**Approach highlights:**
* Build an array of suffix maxima so each index knows the tallest bar to its right.
* Maintain the running maximum height seen so far while iterating from left to right.
* At each position, compute the limiting height as the minimum of the left and right maxima and add any positive difference between that limit and the current height to the total.

**Complexity:** Time O(n), Space O(n).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

// We can trap water on a building if and only if there is a building that is having greater height than that one either on left or on right and water stored for that building will be minimum height of both buildings having height greater than that building minus heght of that building

class Solution
{
public:
    int trap(vector<int>& height)
    {
        int n = height.size();
        int water = 0;
        vector<int> suffMax(n);
        int lefMax = -1;
        suffMax[n - 1] = height[n - 1];
        for(int i = n - 2;i >= 0;i--)
        {
            suffMax[i] = max(suffMax[i + 1],height[i]);
        }
        function<int(int)> Relu;
        Relu = [&](int val)
        {
            if(val <= 0)
            {
                return 0;
            }
            else
            {
                return val;
            }
        };
        for(int i = 0;i < n;i++)
        {
            lefMax = max(lefMax,height[i]);
            int minHeight = min(lefMax,suffMax[i]);
            water += Relu(minHeight - height[i]);
        }
        return water;
    }
};


### Find the Celebrity

**Problem statement:** Given a matrix matrix where matrix[i][j] == 1 means person i knows person j, determine whether a celebrity exists (someone known by everyone else who knows nobody). Return the celebrity's index or -1 if none exists.

**Example:**
```
matrix =
[
  [0, 1, 1],
  [0, 0, 0],
  [0, 1, 0]
]
Output: 1
```
**Explanation:** Two pointers eliminate non-celebrity candidates by comparing the knowledge relationship at each step. The surviving candidate is then validated by checking that they know nobody and are known by everyone else.

**Approach highlights:**
* Start with two indices at the beginning and end of the list of people.
* If the top index knows the bottom index, discard the top index; if the bottom index knows the top, discard the bottom; otherwise discard both.
* Once a single candidate remains, verify their row has no outgoing edges and their column has edges from every other person.
* Return the candidate index if validation succeeds, otherwise return -1.

**Complexity:** Time O(n + m), Space O(1).


In [ ]:
#include <bits/stdc++.h>
using namespace std;

class Solution
{
public:
    int celebrity(vector<vector<int>>& matrix)
    {
        int n = matrix.size();
        int m = matrix[0].size();
        int top = 0;
        int down = n - 1;
        while(top < down)
        {
            if(matrix[top][down] == 1)
            {
                // top knows down it means top isn't a celeb
                top++;
            }
            else if(matrix[down][top] == 1)
            {
                // down knows top it mean down isn't a celeb
                down--;
            }
            else
            {
                // both doesn't know each other it mean both aren't a celeb
                top++;
                down--;
            }
        }
        if(top > down)
        {
            return -1;
        }
        else
        {
            bool isCeleb = true;
            for(int i = 0;i < m;i++)
            {
                if(matrix[top][i] == 1)
                {
                    // if top knows at least one person it means top isn't a celeb
                    isCeleb = false;
                    break;
                }
            }
            for(int i = 0;i < n;i++)
            {
                if(i == top)
                {
                    // top can't know itself
                    continue;
                }
                if(matrix[i][top] == 0)
                {
                    // if top doesn't known by at least one person it means top isn't a celeb
                    isCeleb = false;
                    break;
                }
            }
            if(isCeleb)
            {
                return top;
            }
            else
            {
                return -1;
            }
        }
    }
};
